# Account Details

## Packages & Parameters

In [1]:
from utils.packages import *
warnings.filterwarnings("ignore")
from tqdm import tqdm
from IPython.display import display
import pprint
import json

In [2]:
temp_file = "config/access_keys.yaml"
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)     
    
data = {}    

## Get Invest Details

In [3]:
def get_invest_details():
    global data
    data["access_token"] = config[data['account_name']]["token"]
    data["accountID"] = config[data['account_name']]["account_id"]
    data["api"] = API(access_token = data["access_token"])
    r = accounts.AccountDetails(data["accountID"])

    data["api"].request(r)
    data["account_data"] = r.response
    data["account_balance"] = float(data["account_data"]["account"]["balance"])

    data["order_amount"] = data["account_balance"] * data["invest_ratio"] * 0.8
    data["order_amount"] = int(np.floor(data["order_amount"]))

    print('------------')
    print(f'account_name      : {data["account_name"]}')  
    print(f'account_balance   : {data["account_balance"]} ')
    print(f'order_amount      : {data["order_amount"]} ')
    print('------------')
    return(data)

In [4]:
data["invest_ratio"] = 12.635353654172
account_list = ['oanda_demo_primary','oanda_demo_1', 'oanda_demo_2']

for acc_name in account_list:
    data['account_name'] = acc_name
    get_invest_details()

------------
account_name      : oanda_demo_primary
account_balance   : 99979.2541 
order_amount      : 1010618 
------------
------------
account_name      : oanda_demo_1
account_balance   : 9.2426 
order_amount      : 93 
------------
------------
account_name      : oanda_demo_2
account_balance   : 9.7059 
order_amount      : 98 
------------


In [5]:
def get_candle_size(data):
    params = {"count": data['candle_count']+1, "granularity": data['candle_granularity']}
    candle_size_data = instruments.InstrumentsCandles(instrument="EUR_USD", params=params)
    data['candle_size_info'] = data["api"].request(candle_size_data)
    
    height_list = []
    for candle in data['candle_size_info']['candles']:
        if candle['complete']:
            height_list.append(np.float(candle['mid']['h']) - np.float(candle['mid']['l']))
    
    data['candle_avg_height'] = np.mean(height_list)
    return(data)

In [6]:
%%time
data['candle_count'] = 5
data['candle_granularity'] = "H4"
data = get_candle_size(data)
print(data['candle_avg_height'])

0.00232833333333334
Wall time: 263 ms


## Get Net PL

In [7]:
def get_net_pl():
    global data
    
    start_date = datetime(year=data['start_date']['year'],
                          month=data['start_date']['month'],
                          day=data['start_date']['date'],
                          hour = data['start_date']['hour'],
                          minute=data['start_date']['min']) - dt.timedelta(hours=8)
    
    end_date = datetime(year=data['end_date']['year'],
                          month=data['end_date']['month'],
                          day=data['end_date']['date'],
                          hour = data['end_date']['hour'],
                          minute=data['end_date']['min']) - dt.timedelta(hours=8)
    
    data["access_token"] = config[data['account_name']]["token"]
    data["accountID"] = config[data['account_name']]["account_id"]
    data["api"] = API(access_token = data["access_token"])
    
    
    request_obj = trans.TransactionList(accountID=data["accountID"], params={"pageSize": 1000})
    data["api"].request(request_obj)
    last_transaction = int(request_obj.response['lastTransactionID'])

    for trans_id in tqdm(reversed(range(1,last_transaction))):
        request_obj = trans.TransactionDetails(accountID=data["accountID"], transactionID = trans_id)
        data["api"].request(request_obj)
        trans_time_val = request_obj.response['transaction']['time']
        trans_date = dt.datetime.strptime(trans_time_val.split(".")[0], "%Y-%m-%dT%H:%M:%S")

        if trans_date >= start_date:
            first_trans_id = trans_id
        else:
            break

    params = {"from": first_trans_id, "to": last_transaction}
    request_obj = trans.TransactionIDRange(accountID=data["accountID"], params=params)
    data["api"].request(request_obj)
    all_transactions = request_obj.response   


    pl_list = []
    for t in all_transactions['transactions']:
        trans_ts = dt.datetime.strptime(t['time'].split(".")[0], "%Y-%m-%dT%H:%M:%S")
        if trans_ts > end_date:
            break
        try:
            pl_val = float(t['pl'])
            if pl_val != 0.0:
                pl_list.append(pl_val)
        except:
            pass

    num_transactions = len(pl_list)
    net_pl = round(np.sum(pl_list),5)

    positive_pls = [i for i in pl_list if i > 0]
    negative_pls = [i for i in pl_list if i < 0]

    num_positive = len(positive_pls)
    num_negative = len(negative_pls)

    sum_positive = round(np.sum(positive_pls),5)
    sum_negative = round(np.sum(negative_pls),5)
    sum_total    = round(sum_positive + abs(sum_negative),5) 

    avg_positive = round(np.mean(positive_pls),5)
    avg_negative = round(np.mean(negative_pls),5)
    
    print('-------------')
    print(f'account_name      : {data["account_name"]}')    
    print(f'start_ts          : {start_date + dt.timedelta(hours=8)}')    
    print(f'end_ts            : {end_date + dt.timedelta(hours=8)}')    
    print('-------------')
    print(f'num_transactions  : {num_transactions}')
    print('==============================')
    print(f'net_pl            : {net_pl}/{sum_total}')  
    print('==============================')
    print('-------------')
    print(f'+ve               : num= {num_positive}    sum= +{sum_positive}    avg= +{avg_positive}')
    print(f'-ve               : num= {num_negative}    sum= {sum_negative}    avg= {avg_negative}')
    print('-------------')

    df = pd.DataFrame(pl_list, columns = ['pls'])
    
    values = list(df['pls'].value_counts().index)
    counts = list(df['pls'].value_counts().values)

    df_report = pd.DataFrame({'values':values,'counts':counts})
    df_report['act_val'] = df_report['values'] * df_report['counts']
    df_report = df_report.sort_values(by='values')
    df_report = df_report.reset_index(drop = True)
    display(df_report)
    print(pl_list)
    print('-------------')
    return(all_transactions, pl_list)

In [8]:
data['start_date'] = {'year':2021, 'month':4, 'date':23, 
                      'hour':0, 'min':1}

data['end_date'] = {'year':2021, 'month':4, 'date':30, 
                    'hour':19, 'min':30}

In [9]:
# account_list = ['oanda_demo_primary']
# account_list = ['oanda_demo_1']
# #account_list = ['oanda_demo_2']
# account_list = ['oanda_demo_primary','oanda_demo_1', 'oanda_demo_2']
account_list = ['oanda_demo_1', 'oanda_demo_2']

for acc_name in account_list:
    data['account_name'] = acc_name
    try:
        all_transactions, pl_list = get_net_pl()
    except KeyboardInterrupt:
        print(f'Manually Stopped')
        break
    except:
        print(f'No Transactions yet in {acc_name}')

11it [00:03,  3.27it/s]


-------------
account_name      : oanda_demo_1
start_ts          : 2021-04-23 00:01:00
end_ts            : 2021-04-30 19:30:00
-------------
num_transactions  : 1
net_pl            : -0.1229/0.1229
-------------
+ve               : num= 0    sum= +0.0    avg= +nan
-ve               : num= 1    sum= -0.1229    avg= -0.1229
-------------


,values,counts,act_val
0,-0.1229,1,-0.1229


[-0.1229]
-------------


61it [00:17,  3.47it/s]


-------------
account_name      : oanda_demo_2
start_ts          : 2021-04-23 00:01:00
end_ts            : 2021-04-30 19:30:00
-------------
num_transactions  : 7
net_pl            : -0.1387/0.7163
-------------
+ve               : num= 3    sum= +0.2888    avg= +0.09627
-ve               : num= 4    sum= -0.4275    avg= -0.10688
-------------


,values,counts,act_val
0,-0.1281,1,-0.1281
1,-0.1175,1,-0.1175
2,-0.1078,1,-0.1078
3,-0.0741,1,-0.0741
4,0.0262,1,0.0262
5,0.1083,1,0.1083
6,0.1543,1,0.1543


[0.1543, -0.0741, 0.1083, -0.1078, -0.1281, 0.0262, -0.1175]
-------------
